In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import autocorrelation_plot
import statsmodels.api as sm
import datetime
import numpy as np

In [2]:
# read csv files

opnametrajectC = pd.read_csv("file:///data2/structured data A-UMC/structured data 2021/APROOF_Opnametraject_Opnametraject_C.csv", encoding = "UTF-16", sep = "|")
opnamedeeltrajectC = pd.read_csv("/data2/structured data A-UMC/Xiaoquins data/upload Remko 30-3-2022/APROOF_Opnamedeeltraject_Opnamedeeltraject_C.csv", sep = ";")
opnametrajectB = pd.read_csv("file:///data2/structured data A-UMC/structured data 2017-2020/APROOF_Opnametraject_Opnametraject_B.csv", encoding = "latin-1", sep = ";")
opnamedeeltrajectB = pd.read_csv("/data2/structured data A-UMC/Xiaoquins data/upload Remko 30-3-2022/APROOF_Opnamedeeltraject_Opnamedeeltraject_B.csv", sep = ";")


covid = pd.read_csv("/data2/Documents/a-proof-zonmw/data/covid/covid_data.tsv", encoding = "UTF-8", sep = "\t")
koppeltabel = pd.read_csv("file:///data2/structured data A-UMC/structured data 2021/APROOF_ABC_koppeltabel_jan_2022.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21,24,26,28,53,59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,19,25,49,54,58,59,61,62,64,65,67,68,70,71,73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21,22,27,29,54,60) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,19,22,25,49,54,58,59,61,62,64,65,67,68,70,71,73,74) have mixed types.Specify dtype option on import or set low_memory=

In [3]:
# get icu data from opnametraject_c and opnametraject_b

# b
icu_b = opnamedeeltrajectB[opnamedeeltrajectB["IsICDeeltraject"] == "Ja"]
icu_b = pd.merge(opnametrajectB, icu_b, how = "inner", left_on="OpnametrajectId", right_on="OpnametrajectId", suffixes=("","_r"))

# c
icu_c = opnamedeeltrajectC[opnamedeeltrajectC["IsICDeeltraject"] == "Ja"]
icu_c = pd.merge(opnametrajectC, icu_c, how = "inner", left_on="OpnametrajectId", right_on="OpnametrajectId", suffixes=("","_r"))

In [5]:
icu = icu_b[["Pseudo_id", "OpnameDatum", "OntslagDatum"]].append(icu_c[["Pseudo_id", "OpnameDatum", "OntslagDatum"]]).drop_duplicates()

In [6]:
#merge covid data and icu data

merged_covid = pd.merge(covid, koppeltabel, left_on = "MDN", right_on = "Patientnummer", how = "inner")[['Pseudo_id', 'MDN', 'Notitiedatum',
       'annotated', 'ADM_lvl', 'ATT_lvl', 'BER_lvl', 'ENR_lvl', 'ETN_lvl',
       'FAC_lvl', 'INS_lvl', 'MBW_lvl', 'STM_lvl']]
icu_covid = pd.merge(icu["Pseudo_id"].drop_duplicates(), merged_covid, left_on = "Pseudo_id", right_on = "Pseudo_id", how = "inner")
icu_covid.head()

,Pseudo_id,MDN,Notitiedatum,annotated,ADM_lvl,ATT_lvl,BER_lvl,ENR_lvl,ETN_lvl,FAC_lvl,INS_lvl,MBW_lvl,STM_lvl
0,301140,1830250,2020-03-30,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,301140,1830250,2020-03-31,False,NaN,NaN,NaN,NaN,0.53418,NaN,NaN,NaN,NaN
2,301140,1830250,2020-03-30,False,0.775838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,301140,1830250,2020-03-30,True,0.000000,NaN,NaN,NaN,1.75000,NaN,NaN,NaN,NaN
4,301140,1830250,2020-03-31,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# calculate daily average level

icu_covid_avg = icu_covid.groupby(["MDN", "Notitiedatum"]).mean().reset_index()

In [15]:
# get in-hospital period

icu_covid_inhospital = icu_covid_avg[icu_covid_avg["MDN"] == ""]
for i in range(len(icu_covid_avg)):
    pseudo_id = int(icu_covid_avg[i:i+1]["Pseudo_id"])
    icu_i = icu[icu["Pseudo_id"] == pseudo_id][["OpnameDatum", "OntslagDatum"]]
    opnamedatum = icu_i["OpnameDatum"].values[0]
    ontslagdatum = icu_i["OntslagDatum"].values[0]
    notitiedatum = icu_covid_avg[i:i+1]["Notitiedatum"].values[0]
#     print("%s %s %s" % (opnamedatum, ontslagdatum, notitiedatum))
    if opnamedatum <= notitiedatum <= ontslagdatum:
        icu_covid_inhospital = icu_covid_inhospital.append(icu_covid_avg[i:i+1])

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [16]:
icu_covid_inhospital

,MDN,Notitiedatum,Pseudo_id,annotated,ADM_lvl,ATT_lvl,BER_lvl,ENR_lvl,ETN_lvl,FAC_lvl,INS_lvl,MBW_lvl,STM_lvl
0,54466,2021-02-08,830326,0.000000,0.371338,NaN,NaN,NaN,NaN,NaN,3.173828,NaN,2.833008
1,54466,2021-02-09,830326,0.000000,2.691406,NaN,NaN,NaN,0.574707,NaN,NaN,NaN,NaN
2,54466,2021-02-10,830326,0.000000,1.396667,NaN,NaN,NaN,0.702637,NaN,NaN,NaN,NaN
3,54466,2021-02-11,830326,0.000000,0.043779,NaN,NaN,NaN,0.833089,NaN,NaN,NaN,NaN
4,54466,2021-02-12,830326,0.000000,0.939581,NaN,NaN,NaN,0.905762,NaN,NaN,NaN,2.311198
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,9964647,2020-04-24,83848,0.142857,2.343840,NaN,2.550781,NaN,1.590294,3.142090,NaN,NaN,3.177734
8130,9964647,2020-04-25,83848,0.000000,1.501758,NaN,NaN,NaN,3.380208,2.904800,1.602539,NaN,NaN
8131,9964647,2020-04-26,83848,0.000000,2.409180,NaN,NaN,NaN,3.187500,2.849089,NaN,NaN,3.140625
8132,9964647,2020-04-27,83848,0.166667,2.501831,NaN,NaN,4.0,NaN,3.229405,NaN,1.99707,3.003906


In [17]:
icf_data = icu_covid_inhospital[["Notitiedatum", "Pseudo_id", "ADM_lvl", "ATT_lvl", "BER_lvl", "ENR_lvl", "ETN_lvl", "FAC_lvl", "INS_lvl", "MBW_lvl", "STM_lvl"]]
icf_data.columns = ['date', 'id', "ADM", "ATT", "BER", "ENR", "ETN", "FAC", "INS", "MBW", "STM"]

In [25]:
# fill in default value

columns = ["ADM", "ATT", "BER", "ENR", "ETN", "FAC", "INS", "MBW", "STM"]

mean_icf = icf_data.mean()
filled_icf = None 

for domain in columns:
    df = pd.DataFrame()
    for pseudo_id in icf_data["id"].drop_duplicates():
        data = icf_data[icf_data["id"] == pseudo_id]
        data["date"] = pd.to_datetime(data["date"])
        data1 = data.set_index("date")[domain].interpolate(method="linear")
        data1 = data1.reset_index()
        i = 0
        while i < len(data) and pd.isna(data1.iloc[i,1]):
            i += 1
        if i == len(data):
            data1[domain] = mean_icf[domain]
        else:
            for j in range(i):
                data1.iloc[j,1] = data1.iloc[i,1] 
        data1["id"] = pseudo_id
        df = df.append(data1)
    df = df.set_index(["id", "date"])
    if filled_icf is None:
        filled_icf = df
    else:
        filled_icf = filled_icf.join(df)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
filled_icf.to_csv("icu_icf.csv", sep=";")